# Capítulo 6 - Padrões de projeto com funções de primeira classe

### Estudo de caso: refatorando Stretegy

### Strategy clássico

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

In [2]:
Customer = namedtuple('Customer', 'name fidelity')

In [3]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

In [4]:
class Order: # o Contexto
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [5]:
class Promotion(ABC): # a Estratégia: uma classe-base abstrata
    @abstractmethod
    def discount(self, order):
        """Devolve o desconto como um valor positivo em dólares"""

In [6]:
class FidelityPromo(Promotion): # primeira Estratégia Concreta
    """5% de desconto para clientes com mil ou mais pontos no programa de fidelidade"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

In [7]:
class BulkItemPromo(Promotion): # segunda estratégia Concreta
    """10% de desconto para cada LineItem com 20 ou mais unidades"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

In [8]:
class LargeOrderPromo(Promotion): #terceira Estratégia Concreta
    """7% de desconto para pedidos com 10 ou mais itens diferentes"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.7
        return 0

In [9]:
joe = Customer('John Doe', 0)

In [10]:
ann = Customer('Ann Smith', 1100)

In [11]:
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [12]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [13]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [14]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [15]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [16]:
long_order = [LineItem(str(item_code), 1, 1.0)
             for item_code in range(10)]

In [17]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 3.00>

In [18]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

### Strategy orientado a função

In [19]:
from collections import namedtuple

In [20]:
Customer = namedtuple('Customer', 'name fidelity')

In [21]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

In [22]:
class Order: # o Contexto
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            dscount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [23]:
def fidelity_promo(order):
    """5% de desconto para cliente com mil ou mais pontos no programa de fidelidade"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

In [24]:
def bulk_item_promo(order):
    """10% de desconto para cada LineItem com 20 ou mais unidades"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

In [25]:
def large_order_promo(order):
    """7% de desconto para pedidos com 10 ou mais itens diferentes"""
    discint_items = {item.product for item in order.cart}
    if len(discint_items) >= 10:
        return order.total() * 0.07
    return 0

In [26]:
joe = Customer('John Doe', 0)

In [27]:
ann = Customer('Ann Smith', 1100)

In [28]:
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [29]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [30]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [31]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [32]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [33]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

In [34]:
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [35]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

### Escolhendo a melhor estratégia: abordagem simples

In [37]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

In [38]:
def best_promo(order):
    """Seleciona o melhor desconto disponível"""
    return max(promo(order) for promo in promos)

In [39]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [40]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [41]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

### Encontrando estratégias em um módulo

In [42]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']

In [43]:
def best_promo(order):
    """Seleciona o melhor desconto possível"""
    return max(promo(order) for promo in promos)

In [50]:
import inspect

In [51]:
# promos = [func for name, func in 
#           inspect.getmembers(promotions, inspect.isfunction)]

In [52]:
# def best_promo(order):
#     """Seleciona o melhor desconto possível"""
#     return max(promo(order) for promo in promos)

### Command

In [55]:
class MacroCommand:
    """Um comando que executa uma lista de comandos"""

    def __init__(self, commands):
        self.commans = list(commands)
    
    def __call__(self):
        for commans in self.commans:
            commans()